In [16]:
# !python /home/jovyan/work/src/set_import_paths.py
import pandas as pd
from savReaderWriter import SavReader

In [17]:
sav_file = "/home/jovyan/project-implicit/data/race_iat/Race_IAT.public.2019.sav"

In [18]:
# df = pd.read_spss(sav_file)

In [19]:
from database_connection import databaseConnection
from google.cloud import bigquery


In [20]:
import re
from datetime import datetime as dt
from savReaderWriter import SavReader

def _get_chunck_cutoffs(file_length):
    interval=10_000
    i0=0
    i1=interval
    c_list=[]
    while i1 <= file_length:
        c = [i0, i1]
        c_list += [c]
        i0 = i1 + 1
        i1 = i0 + (interval - 1)
    c_list = c_list + [[i0, file_length]]
    return c_list


def spss_to_csv(filename):
    with SavReader(filename) as reader:
        header = [re.sub("(\'|b)", "", str(h)) for h in reader.header]
        file_length = len(reader)
        lines = []
        df = pd.DataFrame(columns=header)
        chunck_cutoffs = _get_chunck_cutoffs(file_length)
        for i, chunk in enumerate(chunck_cutoffs):
            for line in reader[chunk[0] : chunk[1]]:
                lines += [line]
            df_tmp = pd.DataFrame(lines, columns=header)
            filename_out = re.sub("\.sav", "_{}.csv".format(i), filename)
            print("LOADING {} of {}: {}".format(
                chunk[1], file_length, filename_out))
            df_tmp.to_csv(filename_out)




In [21]:
# df = spss_to_csv(sav_file)
# df.head()

In [28]:

def csv_to_db(source_file, full_table_id):
    
    db = databaseConnection()
    project_id = full_table_id.split('.')[0]
    dataset_id = full_table_id.split('.')[1]
    table_id = full_table_id.split('.')[2]
    client = bigquery.Client(project=project_id, credentials=db.credentials)
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(full_table_id.split('.')[2])
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.skip_leading_rows = 1
    job_config.autodetect = True
    with open(filename, "rb") as source_file:
        job = client.load_table_from_file(source_file, table_ref, job_config=job_config)
    job.result()  # Waits for table load to complete.
    print("Loaded {} rows into {}:{}.{}.".format(
        job.output_rows, project_id, dataset_id, table_id))



In [29]:
filename = '/home/jovyan/project-implicit/data/race_iat/Race_IAT.public.2019_0.csv'
full_table_id = 'algomosaic-nyc.project_implicit.test_table'


In [30]:
csv_to_db(source_file=filename, full_table_id=full_table_id)


Enter the file path and name for GCP's Service Account credentials file. See for more details:
https://console.cloud.google.com/iam-admin/serviceaccounts?project=<projectId>
The current directory is: /home/jovyan/project-implicit/src.
 /home/jovyan/project-implicit/src/project-implicit.json


Loaded 10000 rows into project_implicit:test_table.
